In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import maxjoshua as mh
import tensorflow as tf
import sklearn.preprocessing

## Load dataset

In [2]:
import sklearn.datasets
X, y = sklearn.datasets.make_regression(n_samples=1000, n_features=100, n_informative=20, n_targets=3)

In [3]:
X.shape, y.shape, y.max(), y.min()

((1000, 100), (1000, 3), 890.3432868571163, -905.0260489677341)

## Feature selection

In [4]:
%%time 
indices, values, num_in, num_out = mh.pretrain_submodels(
    sklearn.preprocessing.scale(X), 
    sklearn.preprocessing.scale(y), 
    num_out=64, n_select=3)

0 (13, 36, 61) [0.1751106379409265, 0.3807359347361707, 0.33852142130250107] 0.8476802131506319
1 (0, 38, 67) [0.29272091067020817, 0.08687852204217952, 0.23324329696333823] 0.8649965792871138
2 (0, 40, 57) [0.12630044897664788, 0.5140845338910286, 0.01843514459735876] 0.8095695728079483
CPU times: user 16.4 s, sys: 579 ms, total: 17 s
Wall time: 16 s


## Training

In [5]:
model = tf.keras.models.Sequential([
    mh.SparseLayerAsEnsemble(
        num_in=num_in, 
        num_out=num_out, 
        sp_indices=indices, 
        sp_values=values,
        sp_trainable=False
    ),
    tf.keras.layers.Dense(
        units=3, use_bias=False,
        kernel_constraint=tf.keras.constraints.NonNeg()
    ),
    mh.InverseTransformer(
        units=3,
        init_bias=y.mean(), 
        init_scale=y.std()
    )
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=3e-4, beta_1=.9, beta_2=.999, epsilon=1e-7, amsgrad=True),
    loss='mean_squared_error'
)

history = model.fit(X, y, epochs=3)

Epoch 1/3


2022-06-21 15:09:27.711725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'scale:0' shape=(3,) dtype=float32>
  <tf.Variable 'bias:0' shape=(3,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
32/32 [==============================] - 1s 3ms/step - loss: 33738.9609
Epoch 2/3
32/32 [==============================] - 0s 4ms/step - loss: 29982.0254
Epoch 3/3
32/32 [==============================] - 0s 4ms/step - loss: 27934.1289


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sparse_layer_as_ensemble (S  (None, 64)               592       
 parseLayerAsEnsemble)                                           
                                                                 
 dense (Dense)               (None, 3)                 192       
                                                                 
 inverse_transformer (Invers  (None, 3)                6         
 eTransformer)                                                   
                                                                 
Total params: 790
Trainable params: 398
Non-trainable params: 392
_________________________________________________________________


In [7]:
y_pred = model.predict(X)

32/32 [==============================] - 0s 4ms/step


In [8]:
y_pred

array([[ 184.90265 ,  200.67508 ,  204.9854  ],
       [-188.81943 ,  -47.495148,  -47.88729 ],
       [ -65.356544,  -73.23414 , -170.81003 ],
       ...,
       [  13.653421,   46.261925,   97.69768 ],
       [  23.007442,  120.31909 ,  154.09547 ],
       [-235.95642 , -177.90695 , -234.62137 ]], dtype=float32)

In [9]:
y

array([[ 282.65435149,  163.4616741 ,   44.87586071],
       [-110.93695604,    4.16417994,  -96.30188665],
       [  94.86185737,  195.70192605,  -83.74503846],
       ...,
       [-368.54650326,  -83.11252373,    4.93540598],
       [ 146.93312752,  144.3842028 ,  137.05236528],
       [   4.86349821, -188.70841048, -217.7661992 ]])

In [19]:
# model.save("test-save")
# model2 = tf.keras.models.load_model("test-save")
# y_pred2 = model2.predict(X)
# y_pred == y_pred2